# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

In [3]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

---

# Etiquetas temporales

In [4]:
auctions['hora'] = auctions['date'].dt.hour
auctions['dia'] = auctions['date'].dt.day
auctions['dia_de_la_semana'] = auctions['date'].dt.dayofweek

---

# Armado de ventanas

In [5]:
for ventana_nro in range(1,8):
    auctions["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (19+ventana_nro)))

---

# Armado de ventanas especiales

In [6]:
auctions["ventana_especial_1"] = (((18) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (23)))
auctions["ventana_especial_2"] = (((19) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (23)))
auctions["ventana_especial_3"] = (((20) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (23)))

auctions["ventana_especial_4"] = (((21) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (26)))
auctions["ventana_especial_5"] = (((22) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (26)))
auctions["ventana_especial_6"] = (((23) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (26)))

---

### De auctions, me quedo con sólo datos que correspondan a la competencia

In [7]:
auctions_competencia = auctions.loc[auctions["device_id"].isin(targets_competencia.index)]

### De auctions, me quedo sólo con dispositivos que no estén en la ventana en la que hay que hacer predicciones

In [8]:
auctions_no_estan_en_v7 = auctions.loc[~(auctions["device_id"].isin(auctions.loc[auctions["ventana_7"]]["device_id"]))]

### De los dispositivos de la competencia, obtengo aquellos que no están en la ventana en la que hay que hacer predicciones

In [9]:
competencia_no_estan_en_la_ultima_ventana = targets_competencia.loc[targets_competencia.index.isin(auctions_no_estan_en_v7["device_id"])]

In [10]:
auctions_no_estan_en_v7_competencia = auctions_no_estan_en_v7.loc[auctions_no_estan_en_v7["device_id"].isin(competencia_no_estan_en_la_ultima_ventana.index)]

---

# Creación de features

## Creación de sets de entrenamiento

In [11]:
# Ventanas de tres dias
Xs = {}
for ventana_nro in range(1,5):
    Xs[ventana_nro] = auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"].drop_duplicates().to_frame().set_index("device_id").copy()
    
# Ventanas especiales
Xs_especiales = {}
for ventana_nro in range(1,4):
    Xs_especiales[ventana_nro] = auctions_no_estan_en_v7.loc[auctions_no_estan_en_v7["ventana_especial_{}".format(ventana_nro)]]["device_id"].drop_duplicates().to_frame().set_index("device_id").copy()

## Creación de sets que se usarán para predecir

In [12]:
# Ventanas de tres dias
Xs_predecir = {}
for ventana_nro in range(4,8):
    Xs_predecir[ventana_nro] = auctions_competencia.loc[auctions_competencia["ventana_{}".format(ventana_nro)]]["device_id"].drop_duplicates().to_frame().set_index("device_id").copy()

# Ventanas especiales
Xs_predecir_especiales = {}
for ventana_nro in range(4,7):
    Xs_predecir_especiales[ventana_nro] = auctions_no_estan_en_v7_competencia.loc[auctions_no_estan_en_v7_competencia["ventana_especial_{}".format(ventana_nro)]]["device_id"].drop_duplicates().to_frame().set_index("device_id").copy()

---

## Función generadora de features en ventanas

In [13]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final, especial=False):
    """El rango se toma como en python, [)"""
    if especial is True:
        for ventana_nro in range(ventana_inicial, ventana_final):
            feature = generador_feature(dataframe.loc[dataframe["ventana_especial_{}".format(ventana_nro)]], ventana_nro)
            destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)
    
    else:
        for ventana_nro in range(ventana_inicial, ventana_final):
            feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
            destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)

---

## Cantidad de apariciones en subastas dentro de la ventana

In [14]:
def cantidad_apariciones_en_subastas(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_apariciones_en_subastas_ventana"})

In [15]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_en_subastas, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, cantidad_apariciones_en_subastas, Xs_especiales, 1, 4, especial=True)

In [16]:
generar_feature_en_ventanas(auctions_competencia, cantidad_apariciones_en_subastas, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, cantidad_apariciones_en_subastas, Xs_predecir_especiales, 4, 7, especial=True)

---

## Cantidad de apariciones discriminando períodos

In [17]:
def cantidad_apariciones_por_hora(dataframe, nro_ventana):
    apariciones_por_hora = dataframe.groupby(["device_id", "hora"]).agg(({"date" : "count"}))
    apariciones_por_hora = apariciones_por_hora.unstack().droplevel(level=0, axis=1)
    for hora in range(0, 24):
        if hora not in list(apariciones_por_hora.columns.values):
            apariciones_por_hora.insert(hora, hora, 0)
    apariciones_por_hora.columns = ["apariciones_en_subastas_en_hora_{}".format(hora) for hora in range(0,24)]
    return apariciones_por_hora

In [18]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_por_hora, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, cantidad_apariciones_por_hora, Xs_especiales, 1, 4, especial=True)

In [19]:
generar_feature_en_ventanas(auctions_competencia, cantidad_apariciones_por_hora, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, cantidad_apariciones_por_hora, Xs_predecir_especiales, 4, 7, especial=True)

# No anda por inconsistencias

In [18]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    dias = dataframe["dia"].drop_duplicates
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().droplevel(level=0, axis=1)
    for dia in range(1, len(apariciones_por_dia.columns) + 1):
        if dia not in list(apariciones_por_dia.columns.values):
            apariciones_por_dia.insert(dia, dia, 0)
    apariciones_por_dia.columns = ["apariciones_en_subastas_dia_{}".format(dia) for dia in range(1,len(apariciones_por_dia.columns) + 1)]
    return apariciones_por_dia

In [19]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_por_dia, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, cantidad_apariciones_por_dia, Xs_especiales, 1, 4, especial=True)

In [32]:
generar_feature_en_ventanas(auctions_competencia, cantidad_apariciones_por_dia, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, cantidad_apariciones_por_dia, Xs_predecir_especiales, 4, 7, especial=True)

---

## Sistema operativo (Android / iOS)

In [20]:
def cantidad_so_registrados(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"ref_type_id" : "nunique"}).rename(columns={"ref_type_id": "cantidad_so_registrados_en_auctions"})

In [21]:
generar_feature_en_ventanas(auctions, cantidad_so_registrados, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, cantidad_so_registrados, Xs_especiales, 1, 4, especial=True)

In [22]:
generar_feature_en_ventanas(auctions_competencia, cantidad_so_registrados, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, cantidad_so_registrados, Xs_predecir_especiales, 4, 7, especial=True)

In [23]:
def sistema_operativo_principal(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"ref_type_id" : (lambda x: round(x.mean(), 0))}).rename(columns={"ref_type_id": "so_mas_usado_auctions"})

In [24]:
generar_feature_en_ventanas(auctions, sistema_operativo_principal, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, sistema_operativo_principal, Xs_especiales, 1, 4, especial=True)

In [25]:
generar_feature_en_ventanas(auctions_competencia, sistema_operativo_principal, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, sistema_operativo_principal, Xs_predecir_especiales, 4, 7, especial=True)

---

## Tiempo medio entre apariciones

In [26]:
def tiempo_medio_entre_apariciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).mean().total_seconds()))}).rename(columns={"date": "tiempo_medio_entre_apariciones_subastas"})

In [27]:
generar_feature_en_ventanas(auctions, tiempo_medio_entre_apariciones, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, tiempo_medio_entre_apariciones, Xs_especiales, 1, 4, especial=True)

In [28]:
generar_feature_en_ventanas(auctions_competencia, tiempo_medio_entre_apariciones, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, tiempo_medio_entre_apariciones, Xs_predecir_especiales, 4, 7, especial=True)

---

## Tiempo máximo entre apariciones

In [29]:
def tiempo_maximo_entre_apariciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).max().total_seconds()))}).rename(columns={"date": "tiempo_maximo_entre_apariciones_subastas"})

In [30]:
generar_feature_en_ventanas(auctions, tiempo_maximo_entre_apariciones, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, tiempo_maximo_entre_apariciones, Xs_especiales, 1, 4, especial=True)

In [31]:
generar_feature_en_ventanas(auctions_competencia, tiempo_maximo_entre_apariciones, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, tiempo_maximo_entre_apariciones, Xs_predecir_especiales, 4, 7, especial=True)

---

## Tiempo mínimo entre apariciones

In [32]:
def tiempo_minimo_entre_apariciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).min().total_seconds()))}).rename(columns={"date": "tiempo_minimo_entre_apariciones_subastas"})

In [33]:
generar_feature_en_ventanas(auctions, tiempo_minimo_entre_apariciones, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, tiempo_minimo_entre_apariciones, Xs_especiales, 1, 4, especial=True)

In [34]:
generar_feature_en_ventanas(auctions_competencia, tiempo_minimo_entre_apariciones, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, tiempo_minimo_entre_apariciones, Xs_predecir_especiales, 4, 7, especial=True)

---

## Cantidad de apariciones por fuente

In [54]:
def cantidad_apariciones_segun_source(dataframe, nro_ventana):
    feature = dataframe.groupby(["device_id", "source_id"])["date"].agg("count").to_frame().unstack().droplevel(level=0, axis=1)
    for source in range(0, 10):
        if source not in list(feature.columns.values):
            feature.insert(source, source, 0)
    feature.columns = ["apariciones_en_source_{}_subastas".format(x) for x in range(10)]
    return feature

In [36]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_segun_source, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, cantidad_apariciones_segun_source, Xs_especiales, 1, 4, especial=True)

In [62]:
generar_feature_en_ventanas(auctions_competencia, cantidad_apariciones_segun_source, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, cantidad_apariciones_segun_source, Xs_predecir_especiales, 4, 7, especial=True)

---

## Tiempo desde la última aparición hasta el fin de la ventana

In [64]:
def tiempo_desde_ult_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "max"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_ultima_aparicion_hasta_fin_ventana_subastas"})

In [65]:
generar_feature_en_ventanas(auctions, tiempo_desde_ult_aparicion_hasta_fin_ventana, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, tiempo_desde_ult_aparicion_hasta_fin_ventana, Xs_especiales, 1, 4, especial=True)

In [66]:
generar_feature_en_ventanas(auctions_competencia, tiempo_desde_ult_aparicion_hasta_fin_ventana, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, tiempo_desde_ult_aparicion_hasta_fin_ventana, Xs_predecir_especiales, 4, 7, especial=True)

---

## Tiempo desde la primera aparición hasta el fin de la ventana

In [67]:
def tiempo_desde_prim_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "min"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_primera_aparicion_hasta_fin_ventana_subastas"})

In [68]:
generar_feature_en_ventanas(auctions, tiempo_desde_prim_aparicion_hasta_fin_ventana, Xs, 1, 5)
generar_feature_en_ventanas(auctions_no_estan_en_v7, tiempo_desde_prim_aparicion_hasta_fin_ventana, Xs_especiales, 1, 4, especial=True)

In [69]:
generar_feature_en_ventanas(auctions_competencia, tiempo_desde_prim_aparicion_hasta_fin_ventana, Xs_predecir, 4, 8)
generar_feature_en_ventanas(auctions_no_estan_en_v7_competencia, tiempo_desde_prim_aparicion_hasta_fin_ventana, Xs_predecir_especiales, 4, 7, especial=True)

---

# Guardado de los sets de entrenamiento

In [70]:
for ventana_nro in range(1, 5):
    Xs[ventana_nro].to_pickle("../../features/auctions_ventana_{}_entrenar.pkl".format(ventana_nro))
    Xs[ventana_nro].to_csv("../../features/auctions_ventana_{}_entrenar.csv".format(ventana_nro))

In [72]:
for ventana_nro in range(1,4):
    Xs_especiales[ventana_nro].to_pickle("../../features/auctions_ventana_{}_entrenar_especiales.pkl".format(ventana_nro))
    Xs_especiales[ventana_nro].to_csv("../../features/auctions_ventana_{}_entrenar_especiales.csv".format(ventana_nro))

In [73]:
for ventana_nro in range(4, 8):
    Xs_predecir[ventana_nro].to_pickle("../../features/auctions_ventana_{}_para_predecir.pkl".format(ventana_nro))
    Xs_predecir[ventana_nro].to_csv("../../features/auctions_ventana_{}_para_predecir.csv".format(ventana_nro))

In [74]:
for ventana_nro in range(4,7):
    Xs_predecir_especiales[ventana_nro].to_pickle("../../features/auctions_ventana_{}_para_predecir_especiales.pkl".format(ventana_nro))
    Xs_predecir_especiales[ventana_nro].to_csv("../../features/auctions_ventana_{}_para_predecir_especiales.csv".format(ventana_nro))

---